Terminal command to fetch SST data: `wget -r -l1 -nc -t 50 https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/netcdf/`.

This downloads all SST datasets in NOAA's directory (indexed by year and month from January 1854 to April 2024).

In [2]:
import xarray as xr

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
file_pattern = "ersst.v5.{year:04d}{month:02d}.nc"

start_year = 1854
end_year = 2024

yearly_means = []

for year in range(start_year, end_year + 1):
    monthly_data = []
    
    if year == 2024:
        for month in range(1, 5):
            file_path = file_pattern.format(year=year, month=month)
            ds = xr.open_dataset(file_path)
            monthly_data.append(ds['ssta'])
            ds.close()
    else:
        for month in range(1, 13):
            file_path = file_pattern.format(year=year, month=month)
            ds = xr.open_dataset(file_path)
            monthly_data.append(ds['ssta'])
            ds.close()
        
    yearly_ds = xr.concat(monthly_data, dim='time')
    yearly_mean = yearly_ds.mean(dim='time')
    yearly_means.append(yearly_mean)

yearly_means_ds = xr.concat(yearly_means, dim='year')

output_file = "annual_means.nc"
yearly_means_ds.to_netcdf(output_file)

In [56]:
ds0 = xr.open_dataset("annual_means.nc")

In [126]:
yearly_means_ds = xr.open_dataset("annual_means.nc")

start_year = 1854
end_year = 2024
decade_means = []

first_decade_data = yearly_means_ds.sel(year=slice(1854 - 1854, 1859 - 1854))
first_decade_mean = first_decade_data.mean(dim='year')
decade_means.append(first_decade_mean)

for decade_start in range(1860, end_year, 10):
    decade_end = decade_start + 9
    decade_data = yearly_means_ds.sel(year=slice(decade_start - 1854, decade_end - 1854))
    decade_mean = decade_data.mean(dim='year') 
    decade_means.append(decade_mean)

last_decade_data = yearly_means_ds.sel(year=slice(2020 - 1854, 2024 - 1854))
last_decade_mean = last_decade_data.mean(dim='year')
decade_means.append(last_decade_mean)

decade_means_ds = xr.concat(decade_means, dim='decade')
decade_means_ds.coords['lon'] = (decade_means_ds.coords['lon'] + 180) % 360 - 180
decade_means_ds = decade_means_ds.sortby(decade_means_ds.lon)

decade_means_ds = decade_means_ds.reindex(lat=decade_means_ds['lat'][::-1])

output_file = "decade_means.nc"

decade_means_ds.to_netcdf(output_file)

In [127]:
ds = xr.open_dataset("decade_means.nc")

In [128]:
ds

<xarray.Dataset>
Dimensions:  (lat: 89, lev: 1, lon: 180, decade: 19)
Coordinates:
  * lat      (lat) float64 88.0 86.0 84.0 82.0 80.0 ... -82.0 -84.0 -86.0 -88.0
  * lev      (lev) float64 0.0
  * lon      (lon) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0
Dimensions without coordinates: decade
Data variables:
    ssta     (decade, lev, lat, lon) float32 ...

In [129]:
latitude_bounds = [-14.249902857461892, 48.232259677999046]
longitude_bounds = [-107.03818115217715, -13.070319058558587]

In [130]:
cropped_data = ds.sel(lat=slice(latitude_bounds[1], latitude_bounds[0]),
                                    lon=slice(longitude_bounds[0], longitude_bounds[1]))

cropped_data = cropped_data.reset_index('lev', drop=True)

cropped_data.to_netcdf("cropped_sst_decade.nc")
cropped_data

<xarray.Dataset>
Dimensions:  (lat: 32, lon: 47, decade: 19, lev: 1)
Coordinates:
  * lat      (lat) float64 48.0 46.0 44.0 42.0 40.0 ... -8.0 -10.0 -12.0 -14.0
  * lon      (lon) float64 -106.0 -104.0 -102.0 -100.0 ... -18.0 -16.0 -14.0
Dimensions without coordinates: decade, lev
Data variables:
    ssta     (decade, lev, lat, lon) float32 ...

In [86]:
cropped_data.to_dict("sst.json")

{'coords': {'lat': {'dims': ('lat',),
   'attrs': {'units': 'degrees_north',
    'long_name': 'Latitude',
    'standard_name': 'latitude',
    'axis': 'Y',
    'comment': 'Uniform grid from -88 to 88 by 2'},
   'data': [48.0,
    46.0,
    44.0,
    42.0,
    40.0,
    38.0,
    36.0,
    34.0,
    32.0,
    30.0,
    28.0,
    26.0,
    24.0,
    22.0,
    20.0,
    18.0,
    16.0,
    14.0,
    12.0,
    10.0,
    8.0,
    6.0,
    4.0,
    2.0,
    0.0,
    -2.0,
    -4.0,
    -6.0,
    -8.0,
    -10.0,
    -12.0,
    -14.0]},
  'lon': {'dims': ('lon',),
   'attrs': {},
   'data': [-106.0,
    -104.0,
    -102.0,
    -100.0,
    -98.0,
    -96.0,
    -94.0,
    -92.0,
    -90.0,
    -88.0,
    -86.0,
    -84.0,
    -82.0,
    -80.0,
    -78.0,
    -76.0,
    -74.0,
    -72.0,
    -70.0,
    -68.0,
    -66.0,
    -64.0,
    -62.0,
    -60.0,
    -58.0,
    -56.0,
    -54.0,
    -52.0,
    -50.0,
    -48.0,
    -46.0,
    -44.0,
    -42.0,
    -40.0,
    -38.0,
    -36.0,
    -34.0,
 

In [132]:
import geojson

def netcdf_to_geojson(nc_file, lon_variable, lat_variable, data_variable, geojson_name):
    """
    Returns: a GeoJSON file created from the data in a structured NetCDF file, with coordinates following an 
    equidistant cylindrical projection (EPSG:4087) and a property storing one specified data variable.
    
    Parameters:
    nc_file: String. Path and name of structured NetCDF file to convert.
    lon_variable: String. Name of coordinate variable used to represent the data on a horizontal basis in the NetCDF file (i.e. 'lon', 'x').
    lat_variable: String. Name of coordinate variable used to represent the data on a vertical basis in the NetCDF file (i.e. 'lat', 'y').
    data_variable: String. Name of data variable of interest from the NetCDF file. Will be retained as a property of the GeoJSON file.
    geojson_name: String. Path and name of desired GeoJSON file output.
    """
    # open dataset
    ds = xr.open_dataset(nc_file)

    # identify key variables
    latitude = ds[lat_variable]
    longitude = ds[lon_variable]
    data_var = ds[data_variable]
    
    # prepare GeoJSON
    geojson_features = []
    
    # loop over all coordinate pairs
    for i in range(int(latitude.min().values.item()), int(latitude.max().values.item()), 2):
        for j in range(int(longitude.min().values.item()), int(longitude.max().values.item()), 2):
            data_values = []
            for k in range(1, 19):
                data_value = ds.sel(decade=k)[data_variable].sel(lon = j, lat = i).values.item()
                if data_value == data_value:  
                    data_values.append(data_value)
                else:
                    data_values.append("")

            # Create a GeoJSON feature for each point
            point = geojson.Point((j, i))
            properties = {data_variable: data_values}
            feature = geojson.Feature(geometry=point, properties=properties)
            geojson_features.append(feature)
    
    print(geojson_features)
    # write GeoJSON to File
    with open(geojson_name, "w") as f:
        geojson.dump(geojson.FeatureCollection(geojson_features), f)
        
        
netcdf_to_geojson("cropped_sst_decade.nc", "lon", "lat", "ssta", "ssta_aggregate.geojson")

[{"geometry": {"coordinates": [-106, -14], "type": "Point"}, "properties": {"ssta": [-0.7534897327423096, -0.6543144583702087, -0.3327878713607788, -0.44433820247650146, -0.14170625805854797, -0.5416020154953003, -0.6197541952133179, -0.45458555221557617, -0.14199435710906982, -0.6633585691452026, -0.5281981229782104, -0.5941951274871826, 0.1824134737253189, 0.14314061403274536, 0.0455467514693737, -0.2241649031639099, -0.3065964877605438, -0.3538980782032013]}, "type": "Feature"}, {"geometry": {"coordinates": [-104, -14], "type": "Point"}, "properties": {"ssta": [-0.7249509692192078, -0.6249890327453613, -0.3236362338066101, -0.431937575340271, -0.11182330548763275, -0.5196911096572876, -0.6018456220626831, -0.43766000866889954, -0.12606021761894226, -0.6494956016540527, -0.5141571760177612, -0.5612573623657227, 0.19844987988471985, 0.15233208239078522, 0.043688349425792694, -0.20645177364349365, -0.30318671464920044, -0.34392455220222473]}, "type": "Feature"}, {"geometry": {"coordina